# **run load_data.ipynb BEFORE running this!**

In [10]:
import pickle
import os.path
import numpy as np
normalized_data_file_name = 'data_normalized'
with open(normalized_data_file_name + '.pickle', 'rb') as handle:
    spectrum, temperature = pickle.load(handle)
    print(f"read data from {normalized_data_file_name}.pickle")
print(f"shape of spectrum data: {spectrum.shape}")
print(f"shape of temperature data: {temperature.shape}")
print()
print(f"there are {temperature.shape[0]} spectrums")
print(f"each spectrum is {spectrum.shape[1]} long, which is number of features")

read data from data_normalized.pickle
shape of spectrum data: (6000, 10000)
shape of temperature data: (6000, 1)

there are 6000 spectrums
each spectrum is 10000 long, which is number of features


In [11]:
indices_file_name = 'cross_validation_resample=2_fold=5'
with open(indices_file_name + '.pickle', 'rb') as handle:
    train_indices, test_indices = pickle.load(handle)
    print(f"got indices from {indices_file_name}.pickle")  
print()
print(f"sets of training indices: {len(train_indices)}")
print(f"number of training indices per set: {len(train_indices[0])}")
print(f"sets of testing indices: {len(test_indices)}")
print(f"number of testing indices per set: {len(test_indices[0])}")

got indices from cross_validation_resample=2_fold=5.pickle

sets of training indices: 10
number of training indices per set: 4800
sets of testing indices: 10
number of testing indices per set: 1200


In [12]:
input_dimension = spectrum.shape[1]
print(f"input dimension is: {input_dimension}")
output_dimension = temperature.shape[1]
print(f"output dimension is: {output_dimension}")

input dimension is: 10000
output dimension is: 1


In [13]:
import torch
import torch.nn as nn
class TripletModel(torch.nn.Module):
    def __init__(self, device, input_dim=input_dimension):
        super().__init__()
        self.relu  = nn.ReLU()
        self.hidden_dim = 500
        self.linear1 = torch.nn.Linear(input_dim, self.hidden_dim)
        self.linear2 = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.linear3 = torch.nn.Linear(self.hidden_dim, 1)
        self.device = device
        self.to(device)
    def forward_triplet(self, x):
        y = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
        return y
    def forward(self, input):
        if(input.dim() == 3): # <- training
            # input is tensor of [x1, x2]
            x1 = input[:,0,:] # anchor
            x2 = input[:,1,:] # positive example
            x3 = input[:,2,:] # negative example
            y1 = self.forward_triplet(x1)
            y2 = self.forward_triplet(x2)
            y3 = self.forward_triplet(x3)
            # change: remove sigmoid, use y result as is
            # y = torch.sigmoid(y)
                # euclidean distance for 1 points in 1-space = absolute value?
            # use abs because, in back-prop stage, sqrt(0) results in NAN loss!
            positive_distance = torch.abs(y2 - y1)
            negative_distance = torch.abs(y3 - y1)
            difference = positive_distance - negative_distance # <- order matter, no absolute value!
            # no relu (and no margin), because:
                # this is not classification, 
                # this implementation doesn't know difference between classes
                # therefore, no guarantee that positive distance larger than negative distance!
            # print(f"TripletNetwork: difference: {difference}")
            return difference
        else: # <- actually using: 
            # print(f"TripletModel: actuall using")
            # input is just x
            y = self.forward_triplet(input)
            return y

In [14]:
import random
# reference: 
# https://medium.com/@Skpd/triplet-loss-on-imagenet-dataset-a2b29b8c2952
# https://discuss.pytorch.org/t/getting-nan-after-first-iteration-with-custom-loss/25929/8
class TripletDataset(torch.utils.data.TensorDataset):
    def __init__(self, X, y):
        # subset_X_IDs: e.g. only use mth to nth examples to train
        self.X, self.y = X, y
        self.indices = range(len(y))
    # length: number of elements in subset of X_IDs
    def __len__(self):
        return len(self.y)
    # get item by index in [m, n]
    def __getitem__(self, index): # assume index lies within subset_X_IDs
        # property: randomly sample two inputs,
        # return: the two inputs, 
        #   and binary label for whethery they have same class
        input_1_id = index # anchor
        input_1 = self.X[input_1_id]
        label_1 = self.y[input_1_id]
        input_2_id = random.choice(self.indices) # positive example
        input_2 = self.X[input_2_id]
        label_2 = self.y[input_2_id]
        input_3_id = random.choice(self.indices) # negative example
        input_3 = self.X[input_3_id]
        label_3 = self.y[input_3_id]
        # do: use torch.sqrt
        # do NOT: use np.sqrt 
        #   if use np, need to move number from torch back to cpu to do calculation!
        positive_distance = torch.abs(label_2 - label_1)
        negative_distance = torch.abs(label_3 - label_1)
        difference = positive_distance - negative_distance # order matter!
        return torch.stack([input_1, input_2, input_3]), difference

In [15]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np

loss_function = nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CalculateMSE():
    def __init__(self, net, n_epochs, batch_size):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data).to(device), 
            torch.Tensor(train_label).to(device))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data).to(device), 
            torch.Tensor(test_label).to(device))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        train_loss = []
        validate_loss = []
        criterion = loss_function
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        
        for epoch in range(0, self.n_epochs):
            # if epoch % 1000 == 0:
            #     print(f"epoch = {epoch}")
            epoch_train_loss=[]
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.net.device))
                loss = criterion(y_pred, label.to(self.net.device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            train_loss.append(np.mean(epoch_train_loss))
            # validation step: need to UN-normalize data!
            epoch_loss=[]

            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs_validate, label_validate = data
                    y_pred_validate = self.net(inputs_validate.to(self.net.device))
                    loss_validate = criterion(y_pred_validate, label_validate.to(self.net.device)) # one number
                    epoch_loss.append(loss_validate.item())
            validate_loss.append(np.mean(epoch_loss))
        return np.min(validate_loss), self.net

In [16]:
from pathlib import Path
from datetime import datetime
# change: turn into 10 right now for development, was 3000
n_epochs=1600
batch_size=32

PATH = 'model_triplet/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# change: commented out: alraedy read in X, y data earlier
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
# change: input dim = 1000
mdl = TripletModel(device=device, input_dim=input_dimension)
losses = []
print(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {mdl.device}")

f = open("log_triplet.txt", "w")
f.write("train START: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.write(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {device}\n")
for i,(train,test) in enumerate(zip(train_indices,test_indices)):
    print(f"we are on fold no.{i}")
    train_data, train_label= spectrum[train],temperature[train]
    test_data, test_label= spectrum[test],temperature[test]
    mse_calculator = CalculateMSE(mdl,n_epochs,batch_size)
    loss,model = mse_calculator.get_mse(train_data, 
                                        train_label, 
                                        test_data, 
                                        test_label)
    losses.append(loss)
    print(f"\tloss: {loss}")
    print("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    f.write(f"fold = {i}")
    f.write(f"\tloss: {loss}\n")
    f.write("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") +"\n")

    torch.save(model.state_dict(), PATH+'model_'+str(i))
f.write("train END: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.close()

number of epochs: 1, batch size: 32, device: cuda
we are on fold no.0
	loss: 0.20824053742595622
	time: 2023-05-20 20:02:29
we are on fold no.1
	loss: 0.1997885865134162
	time: 2023-05-20 20:02:30
we are on fold no.2
	loss: 0.24248080620089094
	time: 2023-05-20 20:02:32
we are on fold no.3
	loss: 0.26822806572591934
	time: 2023-05-20 20:02:33
we are on fold no.4
	loss: 0.2619418101133527
	time: 2023-05-20 20:02:34
we are on fold no.5
	loss: 0.23122682583493157
	time: 2023-05-20 20:02:35
we are on fold no.6
	loss: 0.21835697603386803
	time: 2023-05-20 20:02:37
we are on fold no.7
	loss: 0.28367034567369
	time: 2023-05-20 20:02:38
we are on fold no.8
	loss: 0.2049860901929237
	time: 2023-05-20 20:02:39
we are on fold no.9
	loss: 0.23269604126343857
	time: 2023-05-20 20:02:40


In [17]:
losses_mean = np.mean(losses)
losses_std = np.std(losses)
print(f"mean losses: {losses_mean}, std: {losses_std}")

mean losses: 0.23516160849783874, std: 0.027217185554673163


In [18]:
number_figures = 10
import matplotlib.pyplot as plt

indices = torch.randint(0, len(spectrum),(number_figures,)).unique()
for i in indices:
    print(f"we use {i}th example")
    # change: cast i to int, since pandas not work with torch.int64
    # changed: removed figure, since the output is just one number
    spec = np.asarray(spectrum[int(i)]).flatten()
    temp = np.asarray(temperature[int(i)]).flatten()

    prediction = model(torch.Tensor(np.asarray(spectrum[int(i)])).to(model.device)).detach().cpu().flatten()
    
    prediction = prediction.item()
    ground_truth = temp.item()
    # recover data from normalization
    print(f"-----------------------------------------------------")
    print(f"\tthe prediction is: {prediction}")
    print(f"\tthe ground truth is: {ground_truth}")
    print(f"\tthe difference is: {prediction - ground_truth}")

we use 297th example
-----------------------------------------------------
	the prediction is: 20.477659225463867
	the ground truth is: 20.7
	the difference is: -0.2223407745361321
we use 1637th example
-----------------------------------------------------
	the prediction is: 20.971202850341797
	the ground truth is: 21.0
	the difference is: -0.028797149658203125
we use 1816th example
-----------------------------------------------------
	the prediction is: 20.746328353881836
	the ground truth is: 21.0
	the difference is: -0.25367164611816406
we use 2012th example
-----------------------------------------------------
	the prediction is: 21.4593563079834
	the ground truth is: 21.2
	the difference is: 0.25935630798339915
we use 3153th example
-----------------------------------------------------
	the prediction is: 21.63899040222168
	the ground truth is: 21.5
	the difference is: 0.1389904022216797
we use 3739th example
-----------------------------------------------------
	the prediction 

# **skip cell 14 for now (it's in template file!)**